## AS02: Representação Textual

**Aluno: Vinícius Henrique Giovanini**  
**Matricula: 692225**  
**Professor: Wladmir Cardoso Brandão**  

In [1]:
from sklearn.datasets import fetch_20newsgroups
import pprint as pp
import re
import numpy
import json
import pandas as pd

In [2]:
numero_maximo_de_instancias = 10000

In [3]:
newsgroup = fetch_20newsgroups(subset='all', shuffle=False, random_state=42)
pp.pprint(list(newsgroup.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [4]:
len(list(newsgroup.data))

18846

### Funções de Ajuda

In [5]:
def salvar_numpy_array_para_txt(nome_arquivo, array):
    numpy.savetxt(nome_arquivo, array, fmt='%s')


def ler_txt_para_numpy_array(nome_arquivo):
    lista = []

    with open(nome_arquivo, 'r') as arquivo:
        for linha in arquivo:
            lista.append(linha.strip())

    return numpy.array(lista)
  
def salvar_dict_arq(nome_arquivo, dict):
    with open(nome_arquivo, "w") as arquivo:
        for chave, valor in dict.items():
            linha = json.dumps({chave: valor.tolist()}) + '\n'
            arquivo.write(linha)
            
def carregar_dict_arq(nome_arquivo):
    dict_carregado = {}
    with open(nome_arquivo, "r") as arquivo:
        for linha in arquivo:
            obj_json = json.loads(linha)
            chave, valor = list(obj_json.items())[0]
            valor = numpy.array(valor)
            dict_carregado[chave] = valor
    return dict_carregado
  
def salvar_numpy_arq(nome_arquivo, numpy_lista, max_elementos=100):
    with open(nome_arquivo, 'w') as f:
        for array in numpy_lista:
            elementos = array[:max_elementos]  # Apenas os primeiros max_elementos elementos
            f.write("numpy.array([")
            f.write(numpy.array2string(elementos, separator=', ', formatter={'int': lambda x: str(x)}))
            if len(array) > max_elementos:
                f.write(", ...")  # Adiciona "..." se houver mais elementos no array
            f.write("])\n")

In [6]:
def get_tokens(txt):
  tokens = re.sub("[^\w]", " ", txt).split()
  tokens = [w.lower() for w in tokens]
  return tokens

# Pre-processamento
Removendo caracteres especiais 

In [7]:
data_list = list(newsgroup.data)

In [8]:
def clean_txts(txt):
  tokens = re.sub("[^\w]", " ", txt).split()
  clean_txt = [w.lower() for w in tokens]
  clean_txt.append("\n")
  return " ".join(clean_txt)

# Limpando o dataset inteiro removendo caracteres especiais e colocand to lower case
data_list_cleaned = []

for line in data_list:
 data_list_cleaned.append(clean_txts(line))

In [9]:
len(data_list), len(data_list_cleaned)

(18846, 18846)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer()
X_counts = vect.fit_transform(data_list_cleaned)
vocabulario = vect.get_feature_names_out()

In [11]:
len(vocabulario)

173762

In [12]:
salvar_numpy_array_para_txt("../saidas/AS02/data_list_processada.txt", vocabulario)

# One-Hot Encoding


### IMPORTANTE

**OBS**: Rodei o one hot encoding em cima do dataset pre processado, porém tive que limitar o número max de features pois isso estava causando estouro de memória, na saída no txt para não ficar muito pesado para abrir limitei em 500 elementos do array para mostrar

In [13]:
# Criar o vetorizador
vectorizer = CountVectorizer(binary=True, max_features=numero_maximo_de_instancias)
# Ajustar o vetorizador aos textos
vectorizer.fit(data_list_cleaned)
# Transformar os textos em vetores de one-hot encoding
one_hot_encoding = vectorizer.transform(data_list_cleaned).toarray()

In [14]:
one_hot_encoding

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [15]:
salvar_numpy_arq("../saidas/AS02/20News_01.txt", one_hot_encoding, 500)

In [16]:
one_hot_encoding

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# Count Vectors

In [17]:
count_vectorizer = CountVectorizer(max_features=numero_maximo_de_instancias)
count_vectors_resp = count_vectorizer.fit_transform(data_list_cleaned)
count_vectors_array = count_vectors_resp.toarray()

In [18]:
type(count_vectors_array)

numpy.ndarray

In [19]:
salvar_numpy_arq("../saidas/AS02/20News_02.txt", count_vectors_array, 500)

# TF-IDF

In [20]:
tfidf_vectorizer = TfidfVectorizer(max_features=numero_maximo_de_instancias)
tfidf_vectors = tfidf_vectorizer.fit_transform(data_list_cleaned).toarray()

In [21]:
type(tfidf_vectors)

numpy.ndarray

In [22]:
salvar_numpy_arq("../saidas/AS02/20News_03.txt", tfidf_vectors, 500)

# N-grams (2-grams)

In [23]:
count_vectorizer = CountVectorizer(max_features=5000, ngram_range=(1, 2))

count_vectors = count_vectorizer.fit_transform(data_list_cleaned)

count_vectors_array = count_vectors.toarray()

In [24]:
salvar_numpy_arq("../saidas/AS02/20News_04.txt", tfidf_vectors, 500)